<a href="https://colab.research.google.com/github/zzzzzuuuuu/big-data-analytics/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

advertising1,2는 가정별 TV,신문,라디오를 이용하는 시간과 Sales 수치를 나타내서, 어떤 매체를 통해 광고하는 것이 효율적인 지를 분석할 수 있는 데이터입니다. 1번과 2번을 해결하세요.
1번은 넘파이와 판다스만 사용하고, 사이킷런은 사용하지 마세요.

# 1. 데이터 전처리(넘파이, 판다스 기반)

### 1. (a) advertising1.cvs와 advertising2.cvs를 pd.read_csv를 통해서 load하고 advertising의 이름으로 병합하세요, 병합 시에 ignore_index 옵션을 활용하여, 인덱스가 0~199까지 되게 하세요.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

advertising1 = pd.read_csv("./advertising1.csv")
advertising2 = pd.read_csv("./advertising2.csv")

advertising = pd.concat([advertising1, advertising2], ignore_index=True)
advertising

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


### 1. (b) advertising의 각 피처별로 각 피처의 평균값으로 결측치를 채우세요. ex) TV 피처의 평균이 0.5라면 TV 열의 결측치는 0.5로 채우세요.

In [3]:
advertising.fillna({"TV": advertising.TV.mean(),
                    "Radio": advertising.Radio.mean(),
                    "Newspaper": advertising.Newspaper.mean(),
                    "Sales": advertising.Sales.mean()})
# advertising = advertising.fillna(advertising.mean())

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


### 1. (c) advertising이 열 별로 0~1까지의 값을 갖도록, (x-x.min())/(x.max()-x.min())의 최대-최소 스케일링을 적용하세요.

In [5]:
advertising = (advertising - advertising.min(axis=0)) / (advertising.max(axis=0) - advertising.min(axis=0))
# advertising = advertising.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=0)
advertising

,TV,Radio,Newspaper,Sales
0,0.775786,0.762097,0.605981,0.807087
1,0.148123,0.792339,0.394019,0.346457
2,0.055800,0.925403,0.606860,0.409449
3,0.509976,0.832661,0.511873,0.586614
4,0.609063,0.217742,0.510994,0.641732
...,...,...,...,...
195,0.126818,0.074597,0.118734,0.236220
196,0.316199,0.098790,0.068602,0.488189
197,0.596212,0.187500,0.053650,0.519685
198,0.956713,0.846774,0.579595,0.940945


### 1. (d) advertising의 TV, Radio, Newspaper 피쳐에 대해서 각 라벨의 도수가 같도록 세개의 그룹으로로 이산화하여 순서대로 'light', 'middle', 'heavy'로 라벨링하시오.

In [6]:
names = ['light', 'middle', 'heavy']

advertising['TV'] = pd.qcut(advertising['TV'], q=3, labels=names)
advertising['Radio'] = pd.qcut(advertising['Radio'], q=3, labels=names)
advertising['Newspaper'] = pd.qcut(advertising['Newspaper'], q=3, labels=names)

advertising

,TV,Radio,Newspaper,Sales
0,heavy,heavy,heavy,0.807087
1,light,heavy,heavy,0.346457
2,light,heavy,heavy,0.409449
3,middle,heavy,heavy,0.586614
4,middle,light,heavy,0.641732
...,...,...,...,...
195,light,light,light,0.236220
196,light,light,light,0.488189
197,middle,light,light,0.519685
198,heavy,heavy,heavy,0.940945


## 2. 그룹연산 (1번 문제의 advertising 데이터프레임을 계속해서 활용하세요.)

### 2. (a) 아래처럼 advertising의 TV 에 대해서 그룹화를 하고, 평균과 분산에 대해서 'Average', 'Variance'라는 이름으로 집계치를 나타내세요.

In [7]:
tv = advertising.groupby('TV').agg(Average=('Sales', 'mean'), Variance=('Sales', 'var'))
tv

<ipython-input-7-603d4f6082e2>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tv = advertising.groupby('TV').agg(Average=('Sales', 'mean'), Variance=('Sales', 'var'))


,Average,Variance
TV,,
light,0.313374,0.012790
middle,0.553329,0.015929
heavy,0.731696,0.012546


### 2. (b) TV 피처에 대해 그룹화를 진행하고,  샘플별 Sales 값을 그룹별 'Sales' 값의 평균치로 변환하여 나타내세요.

In [10]:
advertising.groupby('TV')['Sales'].transform('mean')
# advertising['Sales'] = advertising.groupby('TV')['Sales'].transform('mean')

<ipython-input-10-d7da776b4904>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  advertising.groupby('TV')['Sales'].transform('mean')


,Sales
0,0.731696
1,0.313374
2,0.313374
3,0.553329
4,0.553329
...,...
195,0.313374
196,0.313374
197,0.553329
198,0.731696


### 2. (c) TV 피처에 대해 그룹화를 진행하고, 그룹별 Sales 값이 가장 큰 10개의 값을 나타내세요.

In [12]:
def top(df, n=10):
  return df.sort_values(ascending=False)[:n]
advertising.groupby('TV')['Sales'].apply(top)

# advertising.groupby('TV').apply(lambda x: x.nlargest(10, 'Sales')[['Sales']])

<ipython-input-12-a25dab1a06b7>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  advertising.groupby('TV')['Sales'].apply(top)


TV         
light   1      0.313374
        140    0.313374
        115    0.313374
        117    0.313374
        119    0.313374
        121    0.313374
        125    0.313374
        126    0.313374
        127    0.313374
        129    0.313374
middle  3      0.553329
        154    0.553329
        105    0.553329
        112    0.553329
        116    0.553329
        118    0.553329
        120    0.553329
        123    0.553329
        139    0.553329
        141    0.553329
heavy   0      0.731696
        147    0.731696
        102    0.731696
        104    0.731696
        109    0.731696
        110    0.731696
        111    0.731696
        113    0.731696
        122    0.731696
        124    0.731696
Name: Sales, dtype: float64

### 2. (d) index는 TV, Radio 피처, columns는 Newspaper가 되고 그룹별 Sales의 평균값을 나타내는 피봇테이블을 나타내세요. (margins 옵션은 True로 하세요.)

In [15]:
advertising.pivot_table(index=["TV", "Radio"], columns=["Newspaper"], values=["Sales"], margins=True)

<ipython-input-15-6aeba7957445>:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  advertising.pivot_table(index=["TV", "Radio"], columns=["Newspaper"], values=["Sales"], margins=True)


Sales                              
Newspaper         light    middle     heavy       All
TV     Radio                                         
light  light   0.313374  0.313374  0.313374  0.313374
       middle  0.313374  0.313374  0.313374  0.313374
       heavy   0.313374  0.313374  0.313374  0.313374
middle light   0.553329  0.553329  0.553329  0.553329
       middle  0.553329  0.553329  0.553329  0.553329
       heavy   0.553329  0.553329  0.553329  0.553329
heavy  light   0.731696  0.731696  0.731696  0.731696
       middle  0.731696  0.731696  0.731696  0.731696
       heavy   0.731696  0.731696  0.731696  0.731696
All            0.525534  0.521409  0.550918  0.532697

## 3. 보스턴 집값 데이터는 집값과 관련된 여러가지 피처와, 보스턴의 집값을 나타내는 데이터입니다. 사이킷런에서 제공하는 방법을 활용해서 아래 스텝에 따라 보스턴 집값 데이터 (boston.csv) 를 분석하세요.
### - 타깃 피처 (집값)은 MEDV입니다.
### - 중요한 파라미터/함수 이외의 파라미터/함수들은 자유롭게 선택해도 됩니다.
### - 각 피처정보는 웹에서 찾아볼 수 있습니다.

### 3. (a) 데이터 준비 및 학습/테스트 데이터 셋 분할을 수행하세요.

In [16]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

boston = pd.read_csv("./boston.csv")

X_train, X_test, y_train, y_test = train_test_split(boston.drop(columns="MEDV"),
                                                    boston.MEDV,
                                                    test_size=0.2,
                                                    random_state=1234)

### 3. (b) 결측치 처리 변환 객체, 스케일링 객체, 라쏘회귀 객체로 이루어진 파이프라인 객체를 생성하세요.

In [18]:
pipeline = Pipeline(steps=[('imputer', SimpleImputer()),
                           ('scaler', StandardScaler()),
                           ('basemodel', SGDRegressor(penalty='l2', random_state=1234, max_iter=10000))])
# 왜 l2지..? 라쏘면 l1 아닌가?

### 3. (c) 러닝레이트 (eta0) 및 정규화 계수 (alpha)에 대해 교차검증 기반 하이퍼파라미터 튜닝을 통해, 최적의 러닝레이트 및 정규화 계수를 알아내세요.  (하이퍼 파라미터 후보군은 자유롭게 고려하세요.)

In [20]:
l_eta0 = [0.0001, 0.001, 0.01, 0.1]
l_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

param_grid = {'basemodel__alpha': l_alpha, 'basemodel__eta0': l_eta0}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)

print('최적 러닝레이트:',grid_search.best_params_['basemodel__eta0'])
print('최적 정규화 계수:',grid_search.best_params_['basemodel__alpha'])

최적 러닝레이트: 0.01
최적 정규화 계수: 0.1


### 3. (d) 테스트 데이터셋에 대해 MAE 성능을 나타내세요.

In [21]:
y_pred_test = grid_search.predict(X_test)
print('테스트 데이터셋 MAE:', np.abs(y_test - y_pred_test).mean())

테스트 데이터셋 MAE: 3.3579240669091193


## 4. 타이타닉 생존 데이터는 각 승객별 여러가지 정보와 생존 여부를 나타내는 데이터입니다. 사이킷런에서 제공되는 방법을 활용해서 아래 스텝에 따라 타이타닉 생존 데이터 (titanic1.csv-titanic4.csv) 를 분석하세요.
### - 타깃 피처 (생존 여부)는 Survived입니다.
### - 중요한 파라미터/함수 이외의 파라미터/함수들은 자유롭게 선택해도 됩니다.
### - 각 피처정보는 웹에서 찾아볼 수 있습니다.

In [44]:
titanic1=pd.read_csv("./titanic1.csv")
titanic2=pd.read_csv("./titanic2.csv")
titanic3=pd.read_csv("./titanic3.csv")
titanic4=pd.read_csv("./titanic4.csv")

# concat으로 합치면 결측치가 생기네,, 왠지모르게,,
titanic_12=pd.concat([titanic1,titanic2],ignore_index=True)
titanic_34=pd.concat([titanic3,titanic4],ignore_index=True)
titanic=pd.merge(titanic_12,titanic_34,on="PassengerId")
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,NaN,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 4. (b) 데이터의 인풋으로 들어가는 피처 중 분석에 필요없는 피처는 제외하고, 피처 중 숫자가 아닌 피처가 있다면 모델을 학습시킬 수 있도록 변환하세요.

In [40]:
titanic = titanic.drop(columns=["Ticket", "Cabin", "Name"])
titanic.Sex = titanic.Sex.map({'male':0, 'female':1})
titanic.Embarked = titanic.Embarked.map({'S':0, 'C':1, 'Q':2})
titanic

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,NaN,0.0
1,2,1,1,1,38.0,1,0,71.2833,1.0
2,3,1,3,1,26.0,0,0,7.9250,0.0
3,4,1,1,1,35.0,1,0,53.1000,0.0
4,5,0,3,0,35.0,0,0,8.0500,0.0
...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.0,0,0,13.0000,0.0
887,888,1,1,1,19.0,0,0,30.0000,0.0
888,889,0,3,1,NaN,1,2,NaN,0.0
889,890,1,1,0,26.0,0,0,30.0000,1.0


### 4. (c) 학습/테스트 데이터셋 분할을 수행하세요.

In [41]:
X_train, X_test, y_train, y_test = train_test_split(titanic.drop(columns="Survived"),
                                                    titanic.Survived,
                                                    test_size=0.2,
                                                    random_state=1234)

### 4. (d) 결측치 처리 변환 객체, 스케일링 객체, 로지스틱 회귀 객체로 이루어진 파이프라인 객체를 생성하세요.

In [42]:
pipeline = Pipeline(steps=[('imputer', SimpleImputer()),
                           ('scaler', StandardScaler()),
                           ('basemodel', LogisticRegression(penalty='l2',
                                                           random_state=1234,
                                                           max_iter=10000,
                                                           solver='sag'))])

### 4. (e) 정규화 계수 (C)에 대해 교차검증 기반 하이퍼파라미터 튜닝을 통해, 최적의 정규화 계수를 알아내세요. (하이퍼 파라미터 후보군은 자유롭게 고려하세요.)

In [43]:
l_C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

param_grid = {'basemodel__C': l_C}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kfold, error_score='raise')
grid_search.fit(X_train, y_train)

print('최적 정규화 계수:',grid_search.best_params_['basemodel__C'])

최적 정규화 계수: 0.01


### 4. (f) 테스트 데이터셋에 대해 classification_report 함수를 통해 분류 성능을 분석하세요.

In [46]:
y_pred_test = grid_search.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       109
           1       0.92      0.64      0.76        70

    accuracy                           0.84       179
   macro avg       0.86      0.80      0.82       179
weighted avg       0.85      0.84      0.83       179

